In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras

In [ ]:
from keras.datasets import cifar10
(X_train , y_train) , (X_test , y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
X_train = X_train.astype(float)/255.0

In [ ]:
X_test = X_test.astype(float)/255.0

In [ ]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [ ]:
model = Sequential()
model.add(Conv2D(24 , (3 , 3), activation='relu',input_shape = (32 , 32 , 3), kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(BatchNormalization())
# model.add(Conv2D(24 , (3 , 3), activation='relu', kernel_initializer='he_uniform' , kernel_regularizer='l2'))
# model.add(BatchNormalization())
model.add(MaxPool2D(2 , 2))
# model.add(Dropout(0.2))
# model.add(Conv2D(48 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(Conv2D(48 , (5 , 5), activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(2 , 2))
# model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(48 , activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dense(10))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train , y_train , epochs = 2 , batch_size=32 , validation_split=0.2)

Epoch 1/2
1250/1250 [==============================] - 43s 33ms/step - loss: 2.4863 - accuracy: 0.4301 - val_loss: 1.8697 - val_accuracy: 0.4962
Epoch 2/2
1250/1250 [==============================] - 41s 33ms/step - loss: 1.6979 - accuracy: 0.5505 - val_loss: 1.7016 - val_accuracy: 0.5271


In [ ]:
model.evaluate(X_test , y_test , verbose = 0)

[1.6859220266342163, 0.531499981880188]

In [ ]:
import tempfile

In [ ]:
_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  /tmp/tmpqm_lbhdi.h5


In [ ]:
pip install tensorflow_model_optimization

     |████████████████████████████████| 174kB 9.6MB/s 


In [ ]:
import tensorflow_model_optimization as tfmot

In [ ]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization
clustering_params = {
  'number_of_clusters': 32,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

In [ ]:
clustered_model = cluster_weights(model, **clustering_params)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 24)        24        
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 24)        96        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 24)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6144)              0         
_________________________________________________________________
dense (Dense)                (None, 48)                48        
_________________________________________________________________
batch_normalization_1 (Batch (None, 48)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [ ]:
clustered_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cluster_conv2d (ClusterWeigh (None, 32, 32, 24)        704       
_________________________________________________________________
cluster_batch_normalization  (None, 32, 32, 24)        96        
_________________________________________________________________
cluster_max_pooling2d (Clust (None, 16, 16, 24)        0         
_________________________________________________________________
cluster_flatten (ClusterWeig (None, 6144)              0         
_________________________________________________________________
cluster_dense (ClusterWeight (None, 48)                294992    
_________________________________________________________________
cluster_batch_normalization_ (None, 48)                192       
_________________________________________________________________
cluster_dense_1 (ClusterWeig (None, 10)                5

In [ ]:
clustered_model.fit(X_train , y_train , epochs = 2 , batch_size=32 , validation_split=0.2)

Epoch 1/2
1250/1250 [==============================] - 41s 33ms/step - loss: 1.6231 - accuracy: 0.5556 - val_loss: 1.6078 - val_accuracy: 0.5655
Epoch 2/2
1250/1250 [==============================] - 40s 32ms/step - loss: 1.5063 - accuracy: 0.6001 - val_loss: 1.5816 - val_accuracy: 0.5725


In [ ]:
clustered_model.evaluate(X_test , y_test , verbose = 0)

[1.5673489570617676, 0.5720999836921692]

In [ ]:
_, keras_file1 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file1)
tf.keras.models.save_model(clustered_model, keras_file1, include_optimizer=False)

Saving model to:  /tmp/tmph9p6zpzn.h5


Normal Model

In [ ]:
print('Saved quantized and clustered TFLite model to:', keras_file)

Saved quantized and clustered TFLite model to: /tmp/tmpqm_lbhdi.h5


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))

Size of gzipped baseline Keras model: 1142203.00 bytes


Normal Model Size

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file1)))

Size of gzipped baseline Keras model: 149515.00 bytes


Clustered Model Size

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
batch_size = 64
epochs = 2
validation_split = 0.175
import numpy as np

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
num_images = 32 * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

In [ ]:
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.1,
                                                               final_sparsity=0.50,
                                                               begin_step=0,
                                                               end_step=end_step)
}

In [ ]:
model_for_pruning = prune_low_magnitude(model, **pruning_params)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [ ]:
model = Sequential()
model.add(Conv2D(24 , (3 , 3), activation='relu',input_shape = (32 , 32 , 3), kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Conv2D(24 , (3 , 3), activation='relu', kernel_initializer='he_uniform' , kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(MaxPool2D(2 , 2))
model.add(Dropout(0.2))
# model.add(Conv2D(48 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(Conv2D(48 , (5 , 5), activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(2 , 2))
# model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(48 , activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(10 , activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train1 = keras.utils.to_categorical(y_train)

In [ ]:
y_test1 = keras.utils.to_categorical(y_test)

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

In [ ]:
model_for_pruning.fit(X_train, y_train1,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
645/645 [==============================] - 107s 160ms/step - loss: 3.4180 - accuracy: 0.3655 - val_loss: 1.9976 - val_accuracy: 0.5410
Epoch 2/2
645/645 [==============================] - 102s 159ms/step - loss: 1.8865 - accuracy: 0.5602 - val_loss: 1.7356 - val_accuracy: 0.5760


In [ ]:
model_for_pruning.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_for_pruning.evaluate(X_test , y_test1 , verbose = 0)

[1.740096926689148, 0.5813000202178955]

In [ ]:
model.fit(X_train , y_train1 , batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/2
645/645 [==============================] - 102s 157ms/step - loss: 3.2765 - accuracy: 0.3805 - val_loss: 2.2359 - val_accuracy: 0.4633
Epoch 2/2
645/645 [==============================] - 101s 157ms/step - loss: 1.8666 - accuracy: 0.5652 - val_loss: 1.6358 - val_accuracy: 0.6145


In [ ]:
model.evaluate(X_test , y_test1 , verbose = 0)

[1.632010817527771, 0.6173999905586243]

In [ ]:
_, keras_file2 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file2)
tf.keras.models.save_model(model, keras_file2, include_optimizer=False)

Saving model to:  /tmp/tmpv5a8hngs.h5


In [ ]:
_, keras_file3 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file3)
tf.keras.models.save_model(model_for_pruning, keras_file3, include_optimizer=False)

Saving model to:  /tmp/tmp7qe2xtqc.h5


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file2)))

Size of gzipped baseline Keras model: 991530.00 bytes


Size of Second Model

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file3)))

Size of gzipped baseline Keras model: 980624.00 bytes


Size of pruned model

In [ ]:
model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_1 (None, 32, 32, 24)        1322      
_________________________________________________________________
prune_low_magnitude_batch_no (None, 32, 32, 24)        97        
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 30, 30, 24)        10394     
_________________________________________________________________
prune_low_magnitude_batch_no (None, 30, 30, 24)        97        
_________________________________________________________________
prune_low_magnitude_max_pool (None, 15, 15, 24)        1         
_________________________________________________________________
prune_low_magnitude_dropout  (None, 15, 15, 24)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 5400)             

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 24)        672       
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 24)        96        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 24)        5208      
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 30, 24)        96        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 24)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 15, 24)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5400)             

In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model

In [ ]:
q_aware_model = quantize_model(model)

In [ ]:
model = Sequential()
model.add(Conv2D(24 , (3 , 3), activation='relu',input_shape = (32 , 32 , 3), kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
model.add(Conv2D(24 , (3 , 3), activation='relu', kernel_initializer='he_uniform' , kernel_regularizer='l2'))
model.add(MaxPool2D(2 , 2))
model.add(Dropout(0.2))
# model.add(Conv2D(48 , (5 , 5), activation='relu', kernel_initializer='he_uniform', padding='same' , kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(Conv2D(48 , (5 , 5), activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(2 , 2))
# model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(48 , activation='relu', kernel_initializer='he_uniform', kernel_regularizer='l2'))
model.add(Dropout(0.4))
model.add(Dense(10 , activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
q_aware_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train , y_train1 , epochs = epochs , validation_split = 0.2 , batch_size=64)

Epoch 1/2
625/625 [==============================] - 79s 125ms/step - loss: 3.0480 - accuracy: 0.1782 - val_loss: 2.1020 - val_accuracy: 0.3354
Epoch 2/2
625/625 [==============================] - 78s 125ms/step - loss: 2.1894 - accuracy: 0.2587 - val_loss: 1.9483 - val_accuracy: 0.3559


In [ ]:
model.evaluate(X_train , y_train1 , verbose = 0)

[1.9432638883590698, 0.36004000902175903]

In [ ]:
q_aware_model.fit(X_train , y_train1 , epochs = 2 , validation_split = 0.15 , batch_size=64)

Epoch 1/2
665/665 [==============================] - 96s 143ms/step - loss: 3.0259 - accuracy: 0.1704 - val_loss: 2.0549 - val_accuracy: 0.3117
Epoch 2/2
665/665 [==============================] - 95s 142ms/step - loss: 2.1433 - accuracy: 0.2645 - val_loss: 1.9092 - val_accuracy: 0.4165


In [ ]:
q_aware_model.evaluate(X_train , y_train1 , verbose = 0)

[1.903838872909546, 0.41850000619888306]

In [ ]:
_, keras_file4 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file4)
tf.keras.models.save_model(model, keras_file4, include_optimizer=False)

Saving model to:  /tmp/tmpip4_x_se.h5


In [ ]:
_, keras_file5 = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file5)
tf.keras.models.save_model(q_aware_model, keras_file5, include_optimizer=False)

Saving model to:  /tmp/tmp02x5i7lb.h5


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file4)))

Size of gzipped baseline Keras model: 1048112.00 bytes


Size of Model 3

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file5)))

Size of gzipped baseline Keras model: 1050633.00 bytes


Size of Quantized model